<a href="https://colab.research.google.com/github/DenisKai7/tobacco/blob/model/Tobacco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import os
import shutil
import pandas as pd

# --- 1. Data Loading and Pre-processing ---


IMAGE_SIZE = (128, 128)
BATCH_SIZE = 32
EPOCHS = 20

# Mount Google Drive (for saving the model)
from google.colab import drive
drive.mount('/content/drive')

# Clone the GitHub repository

print("Cloning the GitHub repository...")
!git clone https://github.com/DenisKai7/tobacco.git


data_dir = '/content/tobacco/datasets/train'
classes_csv_path = os.path.join(data_dir, '_classes.csv')

# Check if the data directory exists
if not os.path.exists(data_dir):
    print(f"Error: Data directory not found at {data_dir}. Please ensure the repository structure is correct.")
    # Attempt to list content for debugging
    print(f"Content of /content/tobacco: {os.listdir('/content/tobacco')}")
    if os.path.exists('/content/tobacco/datasets'):
        print(f"Content of /content/tobacco/datasets: {os.listdir('/content/tobacco/datasets')}")
    exit()

# Read class names from _classes.csv
class_names = []
if os.path.exists(classes_csv_path):
    print(f"Found _classes.csv at: {classes_csv_path}. Reading class names from it.")
    try:

        df_classes = pd.read_csv(classes_csv_path, header=None)
        class_names = df_classes[0].tolist()

        class_names = [name.strip() for name in class_names if name.strip()]
        if not class_names:
            print("Warning: _classes.csv found but appears empty or only whitespace. Falling back to folder names.")
    except Exception as e:
        print(f"Error reading _classes.csv: {e}. Falling back to folder names.")
        class_names = sorted(os.listdir(data_dir))

        if '_classes.csv' in class_names:
            class_names.remove('_classes.csv')
else:
    print(f"No _classes.csv found at: {classes_csv_path}. Inferring class names from subfolder names.")
    class_names = sorted(os.listdir(data_dir))

    if '_classes.csv' in class_names:
        class_names.remove('_classes.csv')


NUM_CLASSES = len(class_names)
if NUM_CLASSES == 0:
    print("Error: No classes found. Please check your dataset directory structure or _classes.csv file.")
    exit()
